In [ ]:
# 避免沒有相關模組
!pip install easyocr google-genai google-api-python-client opencv-python-headless

# Use Module

In [ ]:
# ====================================================================
# I. 核心 AI 服務 (Gemini/RAG) & OCR
# ====================================================================
import cv2                        # 處理圖片 (OCR)
import json                       # 處理 JSON 格式
from google import genai          # Gemini API
from google.genai import types    # Gemini 結構化輸出 Schema
from googleapiclient.discovery import build # Google 搜尋 API
from google.colab import drive # 授權帳號
import easyocr                    # OCR 圖片文字識別
from google.genai.errors import APIError # 處理 Gemini API 錯誤

# ====================================================================
# II. 網路請求 (Web/HTTP) 與解析
# ====================================================================
from bs4 import BeautifulSoup as bs # HTML 解析
import requests as req              # HTTP 請求
from urllib.parse import urljoin, urlparse    # URL 組合

# ====================================================================
# III. 檔案/系統與環境變數管理
# ====================================================================
import os                         # 讀取環境變數
import re                         # 正規表達式
from pathlib import Path as pp    # 檔案路徑操作
import time                       # 執行延遲
from google.colab import userdata  # 載入 Secrets KEY Value：colab環境使用
import copy # 檔案處理
# ====================================================================
# IV. 資料、時間與隨機性
# ====================================================================
import random as rd               # 隨機延遲時間
from datetime import datetime     # 處理日期與時間

# Function


## 展覽基礎資訊爬蟲 - 主要資訊來源

In [ ]:
def baseinfo(pageurl, headers) -> list[dict]:
    baseans = [] # 爬出來資訊儲存用
    # img_path =[] # 圖片路徑
    for ccnt, i in enumerate(pageurl):
        infolist = {
        'strt_dt': None, # 開始日期
        'end_dt': None, # 結束日期
        'name': '', # 展覽名稱
        'article': '', # 重點敘述
        'big_imgurl': '',   # 這裡放圖片的二進制內容，等等每個都放進去辨識內容，取出有用資訊
        'loc': '', # 地點
        'pagetext': '', # 展覽內容敘述
        'pageurl': i, # 展覽內容網址
        'pageimgurl': None
        }

        resi = req.get(i, headers)
        soup = bs(resi.text, 'html.parser')

        infolist['name'] = soup.select_one('div.news_inner p.inner_title').get_text(strip = True)
        infolist['strt_dt']= soup.select_one('div.under > p.montsrt').get_text(strip = True).split(' - ')[0]
        infolist['end_dt']= soup.select_one('div.under > p.montsrt').get_text(strip = True).split(' - ')[1]
        infolist['article']= soup.select_one('article.big_article').get_text(strip = True)
        infolist['big_imgurl']= urljoin(jpgparpath, soup.select_one('img.big_img').get('src'))
        infolist['loc']= soup.select_one('p.place').get_text(strip = True)
        infolist['pagetext']= soup.select_one('section').get_text('\n', strip = True)
        infolist['pageimgurl'] = [imgtag.get('src') for imgtag in soup.select('section img')]

        # col.update_one({'pageurl': infolist['pageurl']}, {'$set': infolist}, upsert = True) # 存進DB
        # col.create_index('pageurl', unique = True) # 設定index
        # col.create_index([('article', 'text'), ('pagetext', 'text')]) # 設定index > 文字索引
        save = (download_img(jpgparpath,
                             infolist['big_imgurl'],
                             save_dir = str(datetime.strftime(datetime.today(), '%Y_%m_%d')) + '_images', # 新建資料匣，以當天日期做資料匣名稱
                             headers = headers,
                             img_nm = infolist['name'])) # 圖片下載，並用標題名稱當作檔名
        for idx, imgurl in enumerate(infolist['pageimgurl']):
            save_inner = (download_img(jpgparpath,
                             imgurl,
                             save_dir = str(datetime.strftime(datetime.today(), '%Y_%m_%d')) + '_images/' + f'{infolist['name']}', # 新建子資料匣，加入展覽名稱做資料匣名稱
                             headers = headers,
                             img_nm = urlparse(imgurl).path.split('/')[-1].split('.')[0]))
            infolist['pageimgurl'][idx] = save_inner if save_inner else None # 置換外部網址成儲存的內部網址
        print(f'[{ccnt + 1}/{len(pageurl)}] {infolist['name']}') # 存進 MongoDB，圖片已下載 {save}
        baseans.append(infolist)
        time.sleep(rd.randint(1, 15))
    return baseans

## 圖片下載

In [ ]:
# ImageDownload Function
def download_img(base_url: str,
                 img_src: str,
                 save_dir: str = 'images',
                 headers: dict = None,
                 img_nm: str = 'unnamed') -> pp | None:
    '''
        base_url (str): 網頁的基準網址，用來處理相對路徑 (例: https://example.com/page.html)
        img_src (str): <img> 標籤裡的 src
        save_dir (str): 存放圖片的資料夾名稱，預設 'images'
        headers (dict): requests 的 HTTP 標頭
    '''

    DEFAULT_USER_AGENT = ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    # 檢查user-agent變數
    if headers is None:
        current_headers = globals().get('hd', {'user-agent': DEFAULT_USER_AGENT})
    else:
        current_headers = headers

    # 把 base_url 和 img_src 合併，確保拿到完整的圖片網址
    img_url = urljoin(base_url, img_src)

    # 用網頁標題當作檔案名稱，排除掉windows非法的檔名符號
    illegal_chars = r'[<>:"/\\|?*．\s]+'
    img_name = img_nm.strip().strip('_')
    filename = re.sub(illegal_chars, '_', img_name) + '.jpg'

    # 建立存放圖片的資料夾 (如果不存在就建立)
    save_path = pp(save_dir)
    save_path = filepath / save_path.with_name(re.sub(illegal_chars, '_', save_path.name))
    save_path.mkdir(parents = True, exist_ok = True)

    # 確認檔案路徑，並檢查是否有重複檔名
    file_path = save_path / filename
    counter = 1
    while file_path.exists():  # 如果檔案已經存在，就在檔名後面加數字避免覆蓋
        stem = file_path.stem      # 不包含副檔名的部分
        suffix = file_path.suffix  # 副檔名 (例如 .jpg)
        file_path = save_path / f'{stem}_{counter}{suffix}'  # 產生新的檔名
        counter += 1

    try:
        resp = req.get(img_url, headers = current_headers, timeout=10)
        resp.raise_for_status()  # 如果狀態碼不是 200，就直接丟出錯誤
        # 把下載回來的二進位內容寫到檔案
        with open(file_path, 'wb') as f:
            f.write(resp.content)
        return file_path  # 成功時回傳圖片存放的完整路徑

    except Exception as e:
        # 如果下載失敗，回傳訊息
        print(f'[{datetime.now()}] {img_url} -> {e}\n')
        return None  # 失敗就回傳 None

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## OCR

In [ ]:
def eocr_process(graph_path: pp, reader: easyocr.Reader) -> str:
    '''
    使用 EasyOCR 讀取多欄位圖片，並將結果格式化為易於 AI 提取的純文字字串
    '''
    if not graph_path.exists():
        return f'Error 圖片檔案未找到於路徑: {graph_path}'

    try:
        # 1. 讀取圖片
        image = cv2.imread(str(graph_path))
        if image is None:
            return 'Error 錯誤：無法讀取圖片，請檢查檔案格式或是否損壞。'

        h, w, _ = image.shape # 圖片shape:高、寬、顏色；第三者為BGR，維度3，但這裡不需要使用

        # 2. 執行 OCR
        results = reader.readtext(image, detail = 1)

    except Exception as e:
        return f'Error EasyOCR 或 OpenCV 發生錯誤: {e}'

    # 2.1 **分左右兩邊區塊** 的動態參數設定================================
    # 設置為圖片高度的 2%。如果文字很小，可能需要降低到 0.01。判斷兩個文字塊是否屬於同一行的 Y 軸距離
    row_tolerance = h * 0.016

    # 如果分隔線不在中間，例如左欄佔 60%，則設為 0.6。判斷文字塊屬於左欄還是右欄的分界線
    center_x_factor = 0.5 # 中間分隔線判斷
    center_x = w * center_x_factor

    processed_data = [] # 用來儲存所有已經確定為一行的數據結構

    # 根據 X 座標分行並標記左右欄位
    for bbox, text, conf in results: # 座標點(共有四點左上[0]、右上[1]、右下[2]、左下[3]順時針)、文字內容、信心程度
        # 取得邊界框的中心點 Y 和 X 座標，並且把每個bbox和text都拿出來判斷
        y_center = (bbox[0][1] + bbox[2][1]) / 2 # 判斷辨識區塊的Y軸中心點
        x_center = (bbox[0][0] + bbox[1][0]) / 2 # 判斷辨識區塊的X軸中心點

        # 根據 X 座標判斷欄位
        column = 'left' if x_center < center_x else 'right' # 如果該判斷區塊，落在中線左邊，則代表是左側的區塊，反之亦然

        # 判斷是否為同一行
        merged = False # 合併開關
        for item in processed_data:
            if abs(y_center - item['y']) < row_tolerance: # 如果Y軸和該輪的區塊Y軸中心小於判斷距離，則判定為同一句話，如果是同一行，將當前的文字塊添加到已存在的item中
                item['texts'].append({'text': text, 'x': x_center, 'col': column, 'y' : y_center})
                merged = True
                break # 找到應該放到哪一行後，則跳出此輪循環

        if not merged: # 如果迴圈結束都找不到匹配的行，則創建一個新的行
            processed_data.append({
                'y': y_center,
                'texts': [{'text': text, 'x': x_center, 'col': column, 'y' : y_center}]
            }) # 紀錄區塊Y軸中心點、辨識出來的文字內容、區塊X軸中心點、判斷後是left還是right

    # 排序與格式化輸出
    processed_data = sorted(processed_data, key = lambda r : r['y']) # 取出 y值進行排序

    left_column_output = ''
    right_column_output = ''

    for row in processed_data:
        # 對同一行列的文字，依 X 座標排序，判定它屬於哪一個行
        left_texts_raw = [t for t in row['texts'] if t['col'] == 'left']
        right_texts_raw = [t for t in row['texts'] if t['col'] == 'right']
        # 先按 X 軸排序，若 X 相同，則用 Y 軸確保垂直順序
        sorted_left_texts = sorted(left_texts_raw, key=lambda t: (t['x'], t['y']))
        sorted_right_texts = sorted(right_texts_raw, key=lambda t: (t['x'], t['y']))

        # 去除空白，並轉為list
        left_texts = [t['text'] for t in sorted_left_texts if t['text'].strip() != '']
        right_texts = [t['text'] for t in sorted_right_texts if t['text'].strip() != '']

        # 將文字各自串接起來，並加上換行符號
        if left_texts:
             left_column_output += ' '.join(left_texts) + '\n'
        if right_texts:
             right_column_output += ' '.join(right_texts) + '\n'

    # 2.2 **不分邊** 的動態參數設定================================
    processed_data_unmerged = [] # 用來儲存所有已經確定為一行的數據結構
    column_output = ''
    # 設置為圖片高度的 1.1%。如果文字很小，可能需要降低到 0.01。判斷兩個文字塊是否屬於同一行的 Y 軸距離
    row_tolerance_unmerged = h * 0.011

    for bbox, text, conf in results: # 座標點(共有四點左上[0]、右上[1]、右下[2]、左下[3]順時針)、文字內容、信心程度
        # 取得邊界框的中心點 Y 和 X 座標，並且把每個bbox和text都拿出來判斷
        y_center = (bbox[0][1] + bbox[2][1]) / 2 # 判斷辨識區塊的Y軸中心點
        x_center = (bbox[0][0] + bbox[1][0]) / 2 # 判斷辨識區塊的X軸中心點

        column = 'center'

        # 判斷是否為同一組
        merged = False # 合併開關
        for item in processed_data_unmerged:
            if abs(y_center - item['y']) < row_tolerance_unmerged: # 如果Y軸和該輪的區塊Y軸中心小於判斷距離，則判定為同一句話，如果是同一行，將當前的文字塊添加到已存在的item中
                item['texts'].append({'text': text, 'x': x_center, 'col': column, 'y' : y_center})
                merged = True
                break # 找到應該放到哪一行後，則跳出此輪循環

        if not merged: # 如果迴圈結束都找不到匹配的行，則創建一個新的行
            processed_data_unmerged.append({
                'y': y_center,
                'texts': [{'text': text, 'x': x_center, 'col': column, 'y' : y_center}]
            }) # 紀錄區塊Y軸中心點
    # 排序與格式化輸出
    processed_data_unmerged = sorted(processed_data_unmerged, key = lambda r : r['y']) # 取出 y值進行排序

    for row in processed_data_unmerged:
        texts_raw = [t for t in row['texts']] # 放到list中
        sorted_texts = sorted(texts_raw, key=lambda t: (t['x'], t['y'])) # 排序，x位置先，y位置後

        # 將文字各自串接起來，並加上換行符號
        sorted_texts_list = [t['text'] for t in sorted_texts if t['text'].strip() != '']
        if sorted_texts_list:
            column_output += ' '.join(sorted_texts_list) + '\n'

    # 最終輸出給 AI 的格式
    ocr_text_output = '--- OCR 圖片內容提取結果（左右分欄）---\n'
    ocr_text_output += '\n=== 左欄內容 (優先閱讀) ===\n' + left_column_output.strip()
    ocr_text_output += '\n\n=== 右欄內容 (次要閱讀) ===\n' + right_column_output.strip()
    ocr_text_output += '\n\n\n=== 不分欄內容 (另外版本) ===\n' + column_output.strip()
    ocr_text_output += '\n--------------------------------------'

    return ocr_text_output

# Main

In [ ]:
cwddate = datetime.strftime(datetime.today(), '%Y%m%d')
print(f'開始抓取，抓取資料日期 {cwddate}')

## 相關路徑設置

In [ ]:
# ====================================================================
# I. 存檔路徑 - google雲端路徑
# ====================================================================
drive.mount('/content/drive')
filepath = pp(r'/content/drive/MyDrive/tibame_proj')

# ====================================================================
# II. 目標網址
# ====================================================================
urlpath = r'https://www.songshanculturalpark.org/exhibition'
jpgparpath = r'https://www.songshanculturalpark.org/gallery/'


## 展覽頁面清單抓取

### 環境設置 - user_agent讀取

In [ ]:
# USER_AGENT = userdata.get('USER_AGENT') # 註解掉原始讀取 secrets 的方式

# 從 Colab Secrets 讀取並設定為環境變數
os.environ['USER_AGENT'] = userdata.get('USER_AGENT')
# 從環境變數中讀取 USER_AGENT
USER_AGENT = os.environ.get('USER_AGENT')


hd = {'user-agent' : USER_AGENT}

### 連結網址抓取

In [ ]:
res = req.get(urlpath, headers = hd)
soup = bs(res.text, 'html.parser')
# 展覽頁面連結
infourl = [urljoin(urlpath, i.find('a', class_='btn')['href']) for i in soup.find_all('span', class_='row_rt')] # 等等要爬的所有網頁連結

## 每個展覽頁面內容

### 文字資訊

In [ ]:
anns = baseinfo(infourl, hd)

### 圖片資訊 - OCR


In [ ]:
try:
    ocr_reader = easyocr.Reader(['ch_tra', 'en']) # 建立模型
    print('Info : EasyOCR 模型載入成功。')

except Exception as e:
    print(f'Error : EasyOCR 模型載入失敗: {e}')
    ocr_reader = None # 如果失敗，將其設置為 None

for item in anns:
    for idx, img in enumerate(item['pageimgurl']):
        print(f'=== OCR辨識中 =========== {idx + 1} / {len(item['pageimgurl'])} ==== {item['name']} {img}') # 讀取進度條，避免我覺得他當掉了
        ocrtext = eocr_process(img, ocr_reader) if img else None
        item['pagetext'] += ('\n接下來是圖片OCR內容文字，' + ocrtext) # 製作要讀取的文本內容

## 用Gemini幫我萃取一下資訊

### 環境設置 - gemini api key value

# 您的專案名稱

## 專案描述
簡要說明您的專案是做什麼的，例如：
這個專案是一個使用 Python 爬取展覽資訊、進行 OCR 文字識別並使用 Gemini API 提取結構化數據的範例專案。

## 環境設定

本專案需要使用到 Google Colab 環境，並依賴以下敏感資訊，這些資訊需要儲存在 Colab 的 Secrets 中：

1.  **`USER_AGENT`**: 用於 HTTP 請求的 User-Agent 字串。
2.  **`GEMINI_API_KEY`**: 您的 Google Gemini API 金鑰，用於文字分析。

請依照以下步驟在您的 Colab 環境中設定 Secrets：

1.  開啟您的 Colab 筆記本。
2.  在左側面板中，點擊「🔑 密鑰」圖標。
3.  點擊「新增密鑰」。
4.  在「名稱」欄位輸入 `USER_AGENT`，在「值」欄位輸入您的 User-Agent 字串。點擊「儲存」。
5.  再次點擊「新增密鑰」。
6.  在「名稱」欄位輸入 `GEMINI_API_KEY`，在「值」欄位輸入您的 Gemini API 金鑰。點擊「儲存」。
7.  確保在筆記本中執行了讀取 Secrets 並設定環境變數的程式碼單元。

In [ ]:
# GEMINI_KEY = userdata.get('GEMINI_API_KEY') # 註解掉原始讀取 secrets 的方式

# 從 Colab Secrets 讀取並設定為環境變數
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
# 從環境變數中讀取 GEMINI_API_KEY
GEMINI_KEY = os.environ.get('GEMINI_API_KEY')

client = genai.Client(api_key = GEMINI_KEY)

exteninfo = [] # 用來儲存提取到的結構化數據
cantcatch = [] # 沒有抓到的資料記錄用
MAX_RETRIES = 5  # 設定最大重試次數
INITIAL_DELAY = 10 # 剛開始的等待秒數

class EmptyResponseError(Exception):
    '''自定義錯誤：當 API 回傳空的文字內容時拋出。'''
    pass

### 統一Schema

In [ ]:
# 定義 結構化產出 (Structured Output): 強制 Gemini在處理一段文字資訊後，必須以一個固定且可預測的 JSON 格式回傳結果，而不是自由形式的文字。
extraction_schema = types.Schema(
    type = types.Type.OBJECT, # 使用 OBJECT 作為單一活動的容器
    properties = {
        'name': types.Schema(type = types.Type.STRING, description = '活動的主要名稱或標題。'),

        'wrktime': types.Schema(type = types.Type.STRING, description = '活動的開放或營業時間，需包含不同日期的變化和最後入場時間的說明，請統一使用 24 小時制，例如 **10:00 - 17:00**。'),

        'price': types.Schema(type = types.Type.STRING, description = '票務或入場資訊，如果免費請寫 **免費入場**。'),

        'note': types.Schema(type = types.Type.STRING, description = '如果展覽是多個項目組成，則將各自的資訊存放於此，需要的內容為**名稱(name)**、**日期(date)**、**時間(wrktime)**、**票價(price)**'),

        'url': types.Schema(
            type = types.Type.ARRAY,
            description = '活動相關的所有重要網址(官網、FB、購票連結等)。',
            items = types.Schema(type = types.Type.STRING, description = '完整的 URL。') # URL 字串的陣列
        )
    },
    required = ['name', 'wrktime', 'price'] # 必要欄位
)

### Prompt


In [ ]:
base_prompt = '''
您是一位專業的數據分析師，以及展覽策展人員。您的任務是從提供的單一文本內容中，識別並嚴格提取所有活動資訊(只要展演活動，不要優惠資訊)。
1.請將提取的結果封裝為單一個 JSON 物件，並遵循我指定的 JSON Schema 格式。
2.請只返回 JSON 格式的內容，不要有任何多餘的解釋或文字。如果只找到部分欄位資訊，就填入這幾個欄位就好，其他填入**無資訊**。
3.內容因為是該展覽負責人員填寫，所以寫法都不同，如果認為文本中沒有相關欄位資訊或內容是空白，則在欄位中填入**無資訊。
4.我會一併給你該展覽提報的簡單資訊，你可以當作參考關鍵字，放在最後並用**{}**包起來。
5.如果遇到名稱中有 **展覽攻略** 頁面，則讀取文本中最上面開頭是 **展演活動** 項下的資訊，看到開頭是**快閃限定、賣店主題、工作坊**等關鍵字，以下內容都排除。
6.遇到展演活動內部有很多活動，請整理後用列點方式放入**備註**欄位中，列點內容包括**名稱**、**開放日期**、**開放時間**、**票價**，不要有任何多餘的解釋或文字


以下是待分析的活動文本，以及最後用{}標住起來的該人員提供的簡單資訊：
'''

# 請模型再思考用
CORRECTION_PROMPT = '''
\n[!!!] 警告：您上一次的輸出無法被解析為有效的 JSON 格式。
請您**嚴格**重新檢查您的輸出內容，並確保它是一個**純淨、完整且符合 JSON 規範**的 JSON 字串
(有沒有可能是少了上下中括弧或是逗點而已，請您注意這點)，請不要包含任何額外的解釋性文字或引言。謝謝！'''

### 文本分析

In [ ]:
for item in anns:
    current_delay = INITIAL_DELAY
    text_content = item['pagetext'] # 頁面上的內容敘述
    curt_name = item['name'] # 頁面上寫的展覽名稱

    # 提示詞外，附上列表上的基礎資訊當作判斷依據 內部RAG工程
    full_prompt = (base_prompt +
                   text_content +
                   '，這個是展覽資訊: {名稱:' + f'{item['name']}、開始時間:{item['strt_dt']}、結束時間:{item['end_dt']}、展覽說明:{item['article']}、地點{item['loc']}' + '}')

    for attempt in range(MAX_RETRIES): # 開始抓取資料，每筆資料最大次數就是5次，超過就送回錯誤
        try:
            print(f'Info : 開始嘗試提取「{curt_name}」活動資訊 (第 {attempt + 1}/{MAX_RETRIES} 次)')

            response = client.models.generate_content(
                model = 'gemini-2.5-flash-lite', # 使用模型 針對速度和效率進行了優化，適用於較輕量級或對延遲要求高的任務
                contents = full_prompt, # 提示詞
                config=types.GenerateContentConfig( # 設定模型如何回應，包括輸出格式、限制和創造性程度等
                    response_mime_type = 'application/json', # 返回json格式資料
                    response_schema = extraction_schema, # 回應的格式按照前面定義的輸出
                    max_output_tokens = 1024, # 限制回傳的token數量，約3-4個英文字母或半個中文字等於1個token
                    temperature = 0.2 # 愈低的值代表模型的回答更具決定性、準確和可預測，適合需要嚴格數據提取和遵循格式的任務。較高的值則適用於寫作、創意或頭腦風暴。
                )
            )

            # 增加一項檢查：確保 response.text 是個字串
            if response is None or response.text is None:
                raise EmptyResponseError(f'Error : API 返回了空的文字內容。')

            # 如果成功，跳出重試循環 ==================================================== 到這步代表有抓到資料
            extracted_json = json.loads(response.text) # dtype dict
            item['extrninfo'] = extracted_json # 將營業時間、票價、官網等資訊補充進去
            print(f'Successed : 「{curt_name}」成功提取：{extracted_json.get('name', '無名稱')}')
            time.sleep(rd.randint(5, 15))
            break
            # =========================================================================

        except (json.JSONDecodeError, EmptyResponseError) as e: # 例外處理 - 回傳內容為空值
            if attempt < MAX_RETRIES - 1:
                print(f'Waring : 警告：=== 「{curt_name}」 === 模型未返回有效 JSON (錯誤：{e})。這次取得的內容是這些  {response.text}')
                print(f'Action : 要求模型自我修正... 等待 5 秒後重試。')
                full_prompt += CORRECTION_PROMPT # 將修正指令附加到提示詞中，請模型重新思考可能錯誤的地方
                time.sleep(5)
                continue # 繼續下一次重試 (帶著修正提示)
            else:
                print(f'Fail : JSON 格式錯誤已達最大重試次數，跳過此項目。')
                cantcatch.append(curt_name) # 將沒有抓到的記錄起來
                time.sleep(rd.randint(5, 15))
                break # 抓不到啦，盡力了，走吧...
        except APIError as e: # 例外處理 - 追蹤API問題
            # 處理 503 等 API 錯誤
            if attempt < MAX_RETRIES - 1 and 'UNAVAILABLE' in str(e):
                print(f'Error : 伺服器過載 (503 錯誤)。等待 {current_delay} 秒後重試...')
                time.sleep(current_delay)
                current_delay *= 1.5
                continue
            else:
                print(f'Fail : API 呼叫失敗，已達最大重試次數，或發生不可恢復錯誤: {e}')
                cantcatch.append(curt_name)
                time.sleep(rd.randint(5, 15))
                break # 抓不到啦，盡力了，走吧...
        except Exception as e: # 其他未知錯誤處理
            print(f'Error : 「{curt_name}」發生未知錯誤: {e}')
            cantcatch.append(curt_name) # 將沒有抓到的記錄起來
            time.sleep(rd.randint(5, 15))
            break
    print('***************************')
print(f'這些是沒有抓到的展覽: {cantcatch}')

# Save Infomation

In [ ]:
rmlist = ['big_imgurl', 'pageimgurl', 'pagetext', 'extrninfo']

test = copy.deepcopy(anns)
savedata = [] # 新增一個 list 來儲存處理後的資料

for ann in test:
    for k in ['wrktime', 'price', 'note']:
        ann[k] = ann['extrninfo'].get(k, '無資訊')
    for j in rmlist:
        if j in ann: # 檢查鍵是否存在，避免 KeyError
            del ann[j]
    savedata.append(ann) # 將處理後的字典添加到列表中
else:
    print(f'Warning: 跳過 {ann.get("name", "Unnamed item")}，內容沒有extrninfo欄位')

output_filename = f'exhibition_info_{datetime.strftime(datetime.today(), "%Y_%m_%d")}.json'
output_path = filepath / output_filename

with open(output_path, encoding = 'utf-8', mode = 'w') as f:
        json.dump(savedata, f, indent = 4, ensure_ascii = False)

# 原始的列印迴圈可以選擇保留或移除，這裡保留供參考
# for item in processed_data:
#     for key, value in item.items():
#         print(f'{key} : {value}')
#     print('****************************')